# 8. Statistics & supervised machine learning

Explanation vs. prediction

TO DECIDE
- ...

### Text book

- Computer Age Statistical Inference

### Notes

- Check https://doi.org/10.1145/1150402.1150412


## 9.1. Explanation (statistical modeling)

- model fit r^2
- variable transformation
- hypothesis and significance testing
- generalized linear model: linear and logistic regression

https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html

#### Data

In [ ]:
import pandas as pd

In [ ]:
vdem_fh = pd.read_csv('../data/V-Dem/vdem_fh_combined.csv')
vdem = pd.read_csv('../data/V-Dem/vdem_only.csv')

In [ ]:
vdem_fh.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme(style='darkgrid')

In [ ]:
indicators_regression = ['v2x_polyarchy', 'v2x_libdem', 'v2x_partipdem', 'v2x_delibdem', 'v2x_egaldem']

In [ ]:
plt.figure(figsize=[2.5, 2])
sns.boxplot(
    x = 'variable', 
    y = 'value', 
    data = vdem_fh[indicators_regression].melt()
)
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=[.5, 2])
sns.boxplot(
    x = 'variable', 
    y = 'value', 
    data = vdem_fh[['Total Score']].melt()
)

### 9.1.1. Regression

#### Correlations

In [ ]:
fig = sns.pairplot(
    data = vdem_fh[['Total Score', 'v2x_polyarchy', 'v2x_libdem', 'v2x_partipdem', 'v2x_delibdem', 'v2x_egaldem']], 
    #hue = 'v2x_regime', 
    height = 2, 
    kind = 'reg', 
    diag_kind = 'hist', 
    plot_kws = {'scatter_kws': {'alpha': .2}}
)
#fig.savefig('.............vdem_fh_pairplot.png')
plt.show()

#### Ordinary-Least-Squares regression

In [ ]:
standardize = False

In [ ]:
X_vdem_fh = vdem_fh[indicators_regression].to_numpy()
y_vdem_fh = vdem_fh[['Total Score']].to_numpy()

if standardize:
    from sklearn.preprocessing import StandardScaler
    
    X_vdem_fh = StandardScaler().fit_transform(X_vdem_fh)
    y_vdem_fh = StandardScaler().fit_transform(y_vdem_fh)
    
    fit_intercept = False
else:
    fit_intercept = True

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
ols_vdem_fh = LinearRegression(fit_intercept=fit_intercept)

In [ ]:
ols_vdem_fh_fit = ols_vdem_fh.fit(X_vdem_fh, y_vdem_fh)

Goodness of fit (R^2)

In [ ]:
ols_vdem_fh_fit.score(X_vdem_fh, y_vdem_fh)

Intercept

In [ ]:
ols_vdem_fh_fit.intercept_

Coefficients

In [ ]:
ols_vdem_fh_fit.coef_[0]

Significance? Not with sklearn!

#### Statsmodels

In [ ]:
import statsmodels.api as sm

In [ ]:
X_vdem_fh_ = sm.add_constant(X_vdem_fh)

In [ ]:
if standardize:
    ols_vdem_fh_sm = sm.OLS(y_vdem_fh, X_vdem_fh)
else:
    ols_vdem_fh_sm = sm.OLS(y_vdem_fh, X_vdem_fh_) # equivalent to fit_intercept=True (https://stackoverflow.com/questions/70179307/why-is-sklearn-r-squared-different-from-that-of-statsmodels-when-fit-intercept-f/70180217#70180217)
ols_vdem_fh_sm_fit = ols_vdem_fh_sm.fit()

In [ ]:
ols_vdem_fh_sm_fit.summary()

In [ ]:
# goodness of fit (R^2)
ols_vdem_fh_sm_fit.rsquared
#ols_vdem_fh_sm_fit.rsquared_adj

In [ ]:
# intercept and coefficients
ols_vdem_fh_sm_fit.params

In [ ]:
# significance scores
ols_vdem_fh_sm_fit.pvalues

In [ ]:
import numpy as np

In [ ]:
def format_coef(df, coef, pvalues=None):
    df[coef] = df[coef].apply(lambda x: '{0:.2f}'.format(x))
    if pvalues is not None:
        def stars(cell):
            if cell <= .001:
                cell = '***'
            elif cell <= .01:
                cell = '**'
            elif cell <= .1:
                cell = '*'
            else:
                cell = ''
            return cell
        df['pvalues'] = pvalues
        df['stars'] = df['pvalues'].apply(stars)
        df[coef] = df[coef] + df['stars']
        del df['pvalues']
        del df['stars']
    return df

In [ ]:
if standardize:
    vdem_fh_regression = pd.DataFrame(
        data = ols_vdem_fh_sm_fit.params, 
        index = indicators_regression, 
        columns = ['OLS']
    )
else:
    vdem_fh_regression = pd.DataFrame(
        data = ols_vdem_fh_sm_fit.params, 
        index = np.concatenate([['intercept'], indicators_regression]), 
        columns = ['OLS']
    )
vdem_fh_regression = format_coef(df=vdem_fh_regression, coef='OLS', pvalues=ols_vdem_fh_sm_fit.pvalues)
vdem_fh_regression

#### Ridge regression to deal with collinearity

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
ridge_vdem_fh = Ridge(alpha=1., fit_intercept=fit_intercept, random_state=42)

In [ ]:
ridge_vdem_fh_fit = ridge_vdem_fh.fit(X_vdem_fh, y_vdem_fh)

In [ ]:
# goodness of fit
ridge_vdem_fh_fit.score(X_vdem_fh, y_vdem_fh)

In [ ]:
# intercept
ridge_vdem_fh_fit.intercept_

In [ ]:
# coefficients
ridge_vdem_fh_fit.coef_[0]

Significance scores are meaningless with regularization (https://stats.stackexchange.com/questions/224796/why-are-confidence-intervals-and-p-values-not-reported-as-default-for-penalized)

In [ ]:
if standardize:
    vdem_fh_regression['Ridge'] = ridge_vdem_fh_fit.coef_[0] # ridge_vdem_fh_sm_fit.params
else:
    vdem_fh_regression['Ridge'] = np.concatenate([ridge_vdem_fh_fit.intercept_, ridge_vdem_fh_fit.coef_[0]])
vdem_fh_regression = format_coef(df=vdem_fh_regression, coef='Ridge', pvalues=None)
vdem_fh_regression

Ridge regression using statsmodels, just for completeness (requires setting scaling penalties: https://stackoverflow.com/questions/72260808/mismatch-between-statsmodels-and-sklearn-ridge-regression):

In [ ]:
n = vdem_fh.shape[0]
if standardize:
    ridge_vdem_fh_sm = sm.OLS(y_vdem_fh, X_vdem_fh)
    penalty = np.ones(len(indicators_regression)) / n

else:
    ridge_vdem_fh_sm = sm.OLS(y_vdem_fh, X_vdem_fh_)
    penalty = np.concatenate([[0.], np.ones(len(indicators_regression))]) / n

ridge_vdem_fh_sm_fit = ridge_vdem_fh_sm.fit_regularized(alpha=penalty, L1_wt=0.)
ridge_vdem_fh_sm_fit.params

#### Lasso regression to automatically select variables

In [ ]:
alpha_lasso = 1. # if alpha < 1. then Lasso becomes Elastic Net (https://stats.stackexchange.com/questions/319861/how-to-interpret-lasso-shrinking-all-coefficients-to-0)

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso_vdem_fh = Lasso(alpha=alpha_lasso, fit_intercept=fit_intercept, random_state=42)

In [ ]:
lasso_vdem_fh_fit = lasso_vdem_fh.fit(X_vdem_fh, y_vdem_fh)

In [ ]:
# goodness of fit
lasso_vdem_fh_fit.score(X_vdem_fh, y_vdem_fh)

In [ ]:
# intercept
lasso_vdem_fh_fit.intercept_

In [ ]:
# coefficients
lasso_vdem_fh_fit.coef_

In [ ]:
if standardize:
    vdem_fh_regression['Lasso'] = lasso_vdem_fh_fit.coef_
else:
    vdem_fh_regression['Lasso'] = np.concatenate([lasso_vdem_fh_fit.intercept_, lasso_vdem_fh_fit.coef_])
vdem_fh_regression = format_coef(df=vdem_fh_regression, coef='Lasso', pvalues=None)
vdem_fh_regression

Lasso regression using statsmodels, just for completeness (requires setting scaling penalties: https://stackoverflow.com/questions/72260808/mismatch-between-statsmodels-and-sklearn-ridge-regression):

In [ ]:
if standardize:
    lasso_vdem_fh_sm = sm.OLS(y_vdem_fh, X_vdem_fh)
    penalty = np.ones(len(indicators_regression))*alpha_lasso
else:
    lasso_vdem_fh_sm = sm.OLS(y_vdem_fh, X_vdem_fh_)
    penalty = np.concatenate([[0.], np.ones(len(indicators_regression))*alpha_lasso])

lasso_vdem_fh_sm_fit = lasso_vdem_fh_sm.fit_regularized(alpha=penalty, L1_wt=1.)
lasso_vdem_fh_sm_fit.params

#### Ordinary-Least-Squares regression with two variables

In [ ]:
indicators_regression_select = ['v2x_polyarchy', 'v2x_libdem']

In [ ]:
X_vdem_fh_select = vdem_fh[indicators_regression_select].to_numpy()

if standardize:
    X_vdem_fh_select = StandardScaler().fit_transform(X_vdem_fh_select)

In [ ]:
X_vdem_fh_select_ = sm.add_constant(X_vdem_fh_select)

In [ ]:
if standardize:
    ols_vdem_fh_select_sm = sm.OLS(y_vdem_fh, X_vdem_fh_select)
else:
    ols_vdem_fh_select_sm = sm.OLS(y_vdem_fh, X_vdem_fh_select_)
ols_vdem_fh_select_sm_fit = ols_vdem_fh_select_sm.fit()

In [ ]:
ols_vdem_fh_select_sm_fit.summary()

In [ ]:
vdem_fh_regression['OLS (select)'] = np.concatenate([ols_vdem_fh_select_sm_fit.params, np.empty(len(indicators_regression)-len(indicators_regression_select))*np.nan])
vdem_fh_regression = format_coef(df=vdem_fh_regression, coef='OLS (select)', pvalues=np.concatenate([ols_vdem_fh_select_sm_fit.pvalues, np.empty(len(indicators_regression)-len(indicators_regression_select))*np.nan]))
vdem_fh_regression

### 9.1.2. Classification

In [ ]:
indicators_logit = ['v2smgovdom_osp', 'v2smgovfilprc_osp', 'v2smgovsmcenprc_osp', 'v2smonper_osp', 'v2smarrest_osp']

In [ ]:
X_vdem_fh_logit = vdem_fh[indicators_logit].to_numpy()
y_vdem_fh_logit = np.where(vdem_fh['v2x_regime'] <= 1, 0, 1).copy()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logit_vdem_fh = LogisticRegression(penalty='none', C=1., fit_intercept=False)

In [ ]:
logit_vdem_fh_fit = logit_vdem_fh.fit(X_vdem_fh_logit, y_vdem_fh_logit)

In [ ]:
# goodness of fit
logit_vdem_fh_fit.score(X_vdem_fh_logit, y_vdem_fh_logit)

In [ ]:
# intercept
logit_vdem_fh_fit.intercept_

In [ ]:
# coefficients
logit_vdem_fh_fit.coef_[0]

In [ ]:
X_vdem_fh_logit_ = sm.add_constant(X_vdem_fh_logit)

In [ ]:
logit_vdem_fh_sm = sm.Logit(y_vdem_fh_logit, X_vdem_fh_logit_)

In [ ]:
logit_vdem_fh_sm_fit = logit_vdem_fh_sm.fit()

In [ ]:
logit_vdem_fh_sm_fit.summary()

In [ ]:
# goodness of fit
logit_vdem_fh_sm_fit.prsquared

In [ ]:
# intercept and coefficients
logit_vdem_fh_sm_fit.params

In [ ]:
# significance scores
logit_vdem_fh_sm_fit.pvalues

In [ ]:
vdem_fh_classification = pd.DataFrame(
    data = logit_vdem_fh_sm_fit.params, 
    index = np.concatenate([['intercept'], indicators_logit]), 
    columns = ['Logit']
)
vdem_fh_classification = format_coef(df=vdem_fh_classification, coef='Logit', pvalues=logit_vdem_fh_sm_fit.pvalues)
vdem_fh_classification

## 9.2. Prediction (supervised machine learning)

- Out-of-sample testing
- Cross validation
- Data leakage
- Feature selection
- Over- and underfitting

- https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html


BOXES: REFER TO SUPPORT VECTOR MACHINES AND NAIVE BAYES CLASSIFICATION

https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html
https://jakevdp.github.io/PythonDataScienceHandbook/05.05-naive-bayes.html

## 9.3. Decision trees and random forests

https://jakevdp.github.io/PythonDataScienceHandbook/05.08-random-forests.html

## 9.4. Towards deep learning

Check https://doi.org/10.1073/pnas.1218772110

- Convey an understanding of neural networks and deep learning
- Start by extendin features by hashtag/named entity/mention representations
- Use neural network off the sklearn shelf

## 9.5. Model selection

- By interpretability
- By predictive accuracy

#### Notes to be removed before publication

- add cvxopt=1.2.6 to environment.yml -- NOT NECSSARY ANYMORE CAUSE WE DON'T USE method=sqrt_lasso IN statsmodels RIDGE REGRESSION ANYMORE
- https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html#sphx-glr-auto-examples-inspection-plot-linear-model-coefficient-interpretation-py
- ...